# Medicare Analysis #

![alt text](https://www.coriers.com/wp-content/uploads/2019/09/photo-1505751172876-fa1923c5c528.jpeg)

For this analysis we will be looking at the Medicare CMS data set provided by [Kaggle.com](www.kaggle.com). 

This data set contains aggergated hospital and medical claims information for the years between 2012-2015. Its a great data set to get started with when it comes to analysis because it has a decent amount of quantifiable data points as well as dimensional fields. This will allow you to slice and dice the data in many different ways and ask more questons.

In this case we will be focusing on the cost of the DRG Codes by state.

This was also originally developed on [Saturncloud.io](https://www.saturncloud.io/).

## Connecting To BigQuery ##

In order to get to the data we will need to use BigQuery. 

BigQuery is a RESTful web service that enables interactive analysis of massive datasets working in conjunction with Google Storage. It is a serverless Platform as a Service that may be used complementarily with MapReduce [Source](https://en.wikipedia.org/wiki/BigQuery).  

Like most APIs you will need to create credentials in order to use BigQuery. So if you don't already, do make sure to create credentials!


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key
#                  file.
# key_path = "path/to/service_account.json"

credentials = service_account.Credentials.from_service_account_file(
    filename="project_g.json"
 
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

medicare = client.dataset('cms_medicare', project='bigquery-public-data')

### Tables ###

You can look at what tables exist in the cms_medicare data set by using the list_tables() function. 

In [223]:
medicare = client.dataset('cms_medicare', project='bigquery-public-data')
print([x.table_id for x in client.list_tables(medicare)])

['home_health_agencies_2013', 'home_health_agencies_2014', 'hospice_providers_2014', 'hospital_general_info', 'inpatient_charges_2011', 'inpatient_charges_2012', 'inpatient_charges_2013', 'inpatient_charges_2014', 'inpatient_charges_2015', 'nursing_facilities_2013', 'nursing_facilities_2014', 'outpatient_charges_2011', 'outpatient_charges_2012', 'outpatient_charges_2013', 'outpatient_charges_2014', 'outpatient_charges_2015', 'part_d_prescriber_2014', 'physicians_and_other_supplier_2012', 'physicians_and_other_supplier_2013', 'physicians_and_other_supplier_2014', 'physicians_and_other_supplier_2015', 'referring_durable_medical_equip_2013', 'referring_durable_medical_equip_2014']


## Running Queries On BigQuery##

Once you have created a reference to the client you can start running queries using the .query() method. It takes your stadnard SQL dialect. In addition, if you need to test your queries first prior to running them, then you should use there online query editor where you can run queries before putting them into code [as seen here] (https://bigquery.cloud.google.com/). Once you have a query that works you can use the basic set up below to run it. 


In [233]:

# How many nurrsing facilitins in different States 
query1 = """SELECT
state, COUNT(state) as total_facilities
FROM
  `bigquery-public-data.cms_medicare.nursing_facilities_2014`
  GROUP BY 
  state
ORDER BY
  total_facilities DESC
  limit 10"""
query_job = client.query(query1)
results = query_job.result()  # Waits for job to complete.

for row in results:
    print(row)

Row(('TX', 1200), {'state': 0, 'total_facilities': 1})
Row(('CA', 1084), {'state': 0, 'total_facilities': 1})
Row(('OH', 936), {'state': 0, 'total_facilities': 1})
Row(('IL', 709), {'state': 0, 'total_facilities': 1})
Row(('PA', 703), {'state': 0, 'total_facilities': 1})
Row(('FL', 688), {'state': 0, 'total_facilities': 1})
Row(('NY', 618), {'state': 0, 'total_facilities': 1})
Row(('IN', 511), {'state': 0, 'total_facilities': 1})
Row(('MO', 497), {'state': 0, 'total_facilities': 1})
Row(('MI', 424), {'state': 0, 'total_facilities': 1})


## Analyzing States With Above Average Healthcare Cost ##

In this analysis we want to answer a simple question. Which states charge more on average compared to the rest.

We will do this by creating a subquery that first gets the average cost per drg definition. Using this number we will then compare it to each states average cost for the same drg definition. Then we will count the total number of drg definition. Using the total number of cases where the drg definition is greater than divided by the total will give us the percentage of drg definitions the state has over the average.

This can be seen below.

In [227]:
import pandas
# How many nurrsing facilitins in different States 
query1 = """


SELECT
  (sum(CASE
    WHEN avg_cost_drug < average_total_payments THEN 1
  ELSE
  0
END)*1.0/count(*))  perc_over,
count(distinct a.drg_definition) total_drg,
 provider_state
FROM (
  SELECT
    sum(total_discharges*average_total_payments)/sum(total_discharges) AS avg_cost_drug,
    drg_definition
  FROM
    `bigquery-public-data.cms_medicare.inpatient_charges_2014`
  GROUP BY
    drg_definition ) a
INNER JOIN
  `bigquery-public-data.cms_medicare.inpatient_charges_2014` b
ON
  a.drg_definition = b.drg_definition
INNER JOIN
  `bigquery-public-data.cms_medicare.hospital_general_info` c
ON
  b.provider_id = c.provider_id
  group by 
  provider_state

"""
query_job = client.query(query1)
results = query_job.result()  # Waits for job to complete.



perc_over=[]
total_drg=[]
provider_state=[]
for i in results:
    perc_over.append(i[0])
    total_drg.append(i[1])
    provider_state.append(i[2])

state_df = pd.DataFrame({
                    'perc_over': perc_over,
                    'total_drg': total_drg,
                    'state': provider_state,
})
print(state_df.sort_values('perc_over',ascending=False,).head())


    perc_over  total_drg state
45   1.000000        114    AK
43   0.977477        145    HI
48   0.842324        278    DC
31   0.837017        175    VT
1    0.808044        401    CA


## Data Visualization ##

Just getting numbers and state codes is difficult to actually process where the states with low and high costs are. In order to mitigate this infromation overload, we will use a basic map of the USA and color code which states have higher costs than others.

We can do this using libraries such as plotly which have built in charts that you can run.

Please see the code below!

In [234]:
from plotly.offline import init_notebook_mode, iplot

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import chart_studio.plotly as py
from plotly import tools

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
init_notebook_mode(connected=True)
scl = [[0.0, 'rgb(248,255,206)'],[0.2, 'rgb(203,255,205)'],[0.4, 'rgb(155,255,164)'], [0.6, 'rgb(79,255,178)'],[0.8, 'rgb(15,183,132)'], [1, '#008059']]


data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = state_df.state,
        z = state_df.perc_over,
        locationmode = 'USA-states',
        text = state_df.state,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage ")
        )
       ]


layout = dict(
        title = 'Percentage Of DRGs That Charge More Than Average By State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

If you want to read more about data science, data engineering, etc, then please feel free to read the articles below!

[Hadoop Vs Relational Database](http://www.acheronanalytics.com/acheron-blog/hadoop-vs-relational-databases)\
[How Algorithms Can Become Unethical and Biased](http://www.acheronanalytics.com/acheron-blog/how-do-machines-learn-bias-data-science)\
[Top 10 Business Intelligence (BI) Implementation Tips](https://www.theseattledataguy.com/top-10-of-business-intelligence-bi-implementation-tips/)\
[5 Great Big Data Tools For The Future - From Hadoop To Cassandra](https://www.theseattledataguy.com/5-great-big-data-tools-for-the-future-from-hadoop-to-cassandra/)\
[The Interview Study Guide For Data Engineers](https://www.coriers.com/the-interview-study-guide-for-data-engineers/)\
[142 Resources for Mastering Coding Interviews](https://medium.com/better-programming/the-software-engineering-study-guide-bac25b8b61eb)\
[Learning Data Science: Our Top 25 Data Science Courses](https://www.coriers.com/25-of-the-best-data-science-courses-online/)\
[The Best And Only Python Tutorial You Will Ever Need To Watch](https://www.coriers.com/the-last-python-tutorial-you-will-ever-need-to-watch/)\
[Dynamically Bulk Inserting CSV Data Into A SQL Server](https://www.youtube.com/watch?v=2bBsYmQjzSE&t=4s)\
[4 Must Have Skills For Data Scientists](https://www.theseattledataguy.com/4-skills-data-scientist-must-have/)\
[What Is A Data Scientist](https://www.theseattledataguy.com/what-is-a-data-scientist/)

